In [ ]:
# Pynnanote audio and whisper
!pip install https://github.com/pyannote/pyannote-audio/archive/refs/heads/develop.zip
!pip install -U openai-whisper

Install torch+cuda here for faster inference using GPU: https://pytorch.org/get-started/locally/ 

Ensure the following: huggingface-hub version 0.15.1 and changing this line in C:\Users\<UserName>\AppData\Local\Programs\Python\Python39\Lib\site-packages\pyannote\audio\core\model.py from from pytorch_lightning.utilities.cloud_io import load as pl_load to from lightning_fabric.utilities.cloud_io import _load as pl_load

In [ ]:
from pyannote.audio import Pipeline
import torch

pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization@2.1",
                                    use_auth_token="ACCESS_TOKEN_GOES_HERE")

In [ ]:
dz = pipeline('audio_file_here', min_speakers=min_speakers_here, max_speakers=max_speakers_here, num_speakers=num_speakers_here)  

for turn, _, speaker in dz.itertracks(yield_label=True):
    print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")

In [16]:
from pydub import AudioSegment
  
wav_file = AudioSegment.from_wav("audio_file_here")

In [5]:
import whisper
import os
model = whisper.load_model("medium.en")
transcription = model.transcribe("audio_file_here")

In [ ]:
# Find timestamps of speakers with pyannnote.audio, then use whisper to transcribe every timestamp
with open("diarization_v1.txt", "a") as text_file:
    for turn, _, speaker in dz.itertracks(yield_label=True):
        print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")
        if turn.end - turn.start <= 0.8: # filter out speech less than 0.8s
            continue
        start_time = int(turn.start * 1000)
        end_time = int(turn.end * 1000)
        wav_file_slice = wav_file[start_time:end_time]
        wav_file_slice.export("temp.wav", format="wav")
        result = model.transcribe("temp.wav")
        result = result['text']
        os.remove("temp.wav")
        with open("diarization_v1.txt", "a") as text_file:
            text_file.write(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}\n")
            text_file.write(f"{result}\n")

In [19]:
# Transcribe entire audio file with whisper first, then match speakers to timestamps
with open("diarization.txt", "w") as text_file:
    for segment in transcription['segments']:
        start = segment['start']
        end = segment['end']
        id_line = f"{segment['id']}, start: {start}, stop: {end}"
        speakers = []
        for turn, _, speaker in dz.itertracks(yield_label=True):
            if turn.end >= start and turn.start <= end and turn.end - turn.start > 0.8: # filter speech less than 0.8s
                speakers.append(speaker)
            if turn.start > end:
                break
        speakers = set(speakers)
        text_file.write(f"{id_line}, {str(speakers) if speakers else 'unidentified'}\n")
        text_file.write(f"\t{segment['text']}\n")

In [ ]:
dz